# Problema de Busca - Super Mario Bros
## Projeto da disciplina **SCC0230 - Inteligência Artificial**
### Assunto: Busca Informada e Busca Cega

## Membros

* Alexandre Eduardo de Souza Jesus - alexandre_souza@usp.br - 12559506

* Arthur Santorum Lorenzetto - arts.lorenzetto@usp.br -

* Eduardo Zaffari Monteiro - eduardozaffarimonteiro@usp.br - 

* Gustavo Silva de Oliveira - gustavo.oliveira03@usp.br - 12567231 (Vice-Líder)

* Lucas Ivars Cadima Ciziks - luciziks@usp.br - 125599472 (Líder)

* Pedro Henrique de Freitas Maçonetto - pedromaconetto@usp.br - 

* Vitor Okubo Ianella - vitorok.03@gmail.com - 

## Introdução

* Motivação = Descrever o objetivo do trabalho;
* Especificação = Explicar a temática do problema e ferramentas utilizadas;
* Contextualização = Resumir as estratégias abordadas.

## Modelagem do Problema

Descrever os seguintes aspectos do problema:
* Estados;
* Operadores;
* Movimentos válidos;
* Configuração final.

Detalhar visualmente o Espaço de Estados (Matriz Esparsa)

In [7]:
import numpy as np
import pandas as pd

In [8]:
labirinto = np.genfromtxt('labirinto.csv', delimiter=',')
labirinto

array([[0., 0., 0., 0., 0., 0., 0.],
       [0., 3., 1., 1., 1., 1., 0.],
       [0., 1., 0., 0., 0., 1., 0.],
       [0., 1., 0., 1., 1., 1., 0.],
       [0., 1., 1., 1., 0., 4., 0.],
       [0., 0., 0., 0., 0., 0., 0.]])

## Busca Não Informada

* Justificar escolha do algoritmo;
* Especificar decisões de implementação: Estruturas de Dados, manipulação dos dados, adaptações ao problema, etc.

In [37]:
# Implementar a Busca Cega do Trabalho (Busca em Profundidade)
# Em Profundidade: https://www.geeksforgeeks.org/depth-first-search-or-dfs-for-a-graph/?ref=lbp
# Implementar a Busca Cega do Trabalho (Busca em Profundidade ou em Largura)
# Em Largura: https://www.geeksforgeeks.org/breadth-first-search-or-bfs-for-a-graph/
# Em Profundidade: https://www.geeksforgeeks.org/depth-first-search-or-dfs-for-a-graph/?ref=lbp
import pandas as pd
import numpy as np
from collections import deque

lab = pd.read_csv('labirinto.csv')
lab = lab.to_numpy()

def get_values():
    
    start = np.where(lab==3)
    i = int(start[0])
    j = int(start[1])
    start = (i,j)
    
    end = np.where(lab==4)
    i = int(end[0])
    j = int(end[1])
    end = (i,j)
    
    cogu = np.where(lab==5)
    i = int(cogu[0])
    j = int(cogu[1])
    cogu = (i,j)
    
    goomba = np.where(lab==2)
    i = int(goomba[0])
    j = int(goomba[1])
    goomba = (i,j)

    where_2 = np.where(lab == 2)
    where_3 = np.where(lab == 3)
    where_4 = np.where(lab == 4)
    lab[where_3] = 1
    lab[where_4] = 1
    lab[where_2] = 0
    
    
    return start, end, cogu, goomba

def maze2graph(maze):
    height = len(maze)
    width = len(maze[0]) if height else 0
    graph = {(i, j): [] for j in range(width) for i in range(height) if maze[i][j]}
    for row, col in graph.keys():
        if not(row < height - 1 and not maze[row + 1][col]):
            graph[(row, col)].append(("S", (row + 1, col)))
            graph[(row + 1, col)].append(("N", (row, col)))
        if not(col < width - 1 and not maze[row][col + 1]):
            graph[(row, col)].append(("L", (row, col + 1)))
            graph[(row, col + 1)].append(("O", (row, col)))
    return graph

def find_path_dfs(start, end):
    stack = deque([("", start)])
    visited = set()
    graph = maze2graph(lab)
    while stack:
        path, current = stack.pop()
        if current == end:
            return path
        if current in visited:
            continue
        visited.add(current)
        for direction, neighbour in graph[current]:
            stack.append((path + direction, neighbour))
    return "NO WAY!"

start, end, cogu, goomba = get_values()
start_to_cogu = find_path_dfs(start, cogu)

lab[goomba[0]][goomba[1]] = 1

cogu_to_goomba = find_path_dfs(cogu, goomba)
goomba_to_end = find_path_dfs(goomba, end)

lab_matrix = lab.tolist()
lab_matrix[start[0]][start[1]] = 'S'
lab_matrix[end[0]][end[1]] = 'E'
lab_matrix[cogu[0]][cogu[1]] = 'C'
lab_matrix[goomba[0]][goomba[1]] = 'G'
lab_matrix = [[str(ele) for ele in sub] for sub in lab_matrix]


def make_path(maze, path, start):
    caminho_atual = start
    path = path[0:len(path)-1]
    for value in list(path):
        if value == 'L':
            caminho_atual = (caminho_atual[0], caminho_atual[1]+1)
            maze[caminho_atual[0]][caminho_atual[1]] = '→'
        elif value == 'O':
            caminho_atual = (caminho_atual[0], caminho_atual[1]-1)
            maze[caminho_atual[0]][caminho_atual[1]] = '←'
        elif value == 'N':
            caminho_atual = (caminho_atual[0]-1, caminho_atual[1])
            maze[caminho_atual[0]][caminho_atual[1]] = '↑'
        else:
            caminho_atual = (caminho_atual[0]+1, caminho_atual[1])
            maze[caminho_atual[0]][caminho_atual[1]] = '↓'
        
        return maze

lab_matrix_1 = make_path(lab_matrix,start_to_cogu,start)
lab_matrix_2 = make_path(lab_matrix,cogu_to_goomba,cogu)

lab_matrix
# print(start_to_cogu, cogu_to_goomba, goomba_to_end)
# print(lab)

[['0', '0', '0', '0', '0', '0', '0', '0'],
 ['0', 'S', '1', '1', 'G', '1', '0', '0'],
 ['0', '↓', '0', '0', '0', '1', '0', '0'],
 ['0', '1', '0', 'C', '→', '1', '0', '0'],
 ['0', '1', '1', '1', '0', 'E', '0', '0'],
 ['0', '0', '0', '0', '0', '0', '0', '0']]

## Busca Informada

* Justificar escolha do algoritmo;
* Fundamentar heurística utilizada (Admissibilidade);
* Especificar decisões de implementação: Estruturas de Dados, manipulação dos dados, adaptações ao problema, etc.

In [46]:
# Implementar a Busca Informada do Trabalho
# A*: https://www.geeksforgeeks.org/a-search-algorithm/
# Greedy Best-First: https://www.geeksforgeeks.org/best-first-search-informed-search/


## Conclusão

* Comparar algoritmos com alguns casos de Teste;
* Interpretar resultados obtidos;
* Considerações Finais acerca do problema.

## Referências


BRATKO, I. Prolog Programming for Artificial Intelligence . Mitchell, T. Machine
Learning , McGraw Hill, 1997.

RUSSEL, S.; NORVIG, P. Artificial Intelligence : A Modern Approach. Prentice
Hall; 3 edition (December 11, 2009).

https://www.geeksforgeeks.org/difference-between-informed-and-uninformed-search-in-ai/?ref=lbp

https://www.sciencedirect.com/topics/mathematics/manhattan-distance

https://www.geeksforgeeks.org/difference-between-bfs-and-dfs/?ref=lbp

http://theory.stanford.edu/~amitp/GameProgramming/AStarComparison.html

http://theory.stanford.edu/~amitp/GameProgramming/